# Title:

## Predicting Extend of Heart Disease with Classification

# Introduction:

For my project proposal, I will be using a modified version of the UCI Heart Disease dataset provided by DSCI100. The UCI Heart Disease dataset includes four databases of the countries: Cleveland, Hungary, Switzerland, and the VA Long Beach. I will be using Cleveland for my prediction because the dataframe has the most observations out of the four options, as a larger dataset should lead to a better model. The Heart Disease Cleveland dataframe includes the variables: "trestbps" and "thalach", which I will be using to predict the variable of intrest: "num". In the dataframe, "trestbps" refers to the resting blood pressure on admission to the hospital measured in mm Hg, while "thalach" refers to maximum heart rate achieved measured in bps, and "num" means the diagnosis of heart disease with a value of: 0 means no heart disease, 1-4 refering to various levels of heart disease (Classified as 1: Mild, 2: Moderate, 3: Severe 4: Life threatening). With these resources, I will be answering the question: Can I predict the extent of heart disease through classification using the variables resting blood pressure and maximum heart rate achieved?

# Preliminary Data Analysis:

Demonstrate that the dataset can be read from the web into R
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data.
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co